<a href="https://colab.research.google.com/github/Celest14l/AI-Volution/blob/main/Ai_volution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Requirements

In [3]:
!pip install gradio flask apscheduler plyer



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.3/142.3 kB 5.9 MB/s eta 0:00:00


In [9]:
!pip install openai flask pyotp cryptography


# Task Reminder

In [8]:
from datetime import datetime, timedelta
from apscheduler.schedulers.background import BackgroundScheduler
from plyer import notification

# In-memory task storage
tasks = {}
task_id_counter = 1

# Function to add a task
def add_task(title, due_date, priority):
    global task_id_counter
    try:
        due_date = datetime.strptime(due_date, "%Y-%m-%d %H:%M")
        tasks[task_id_counter] = {
            "title": title,
            "due_date": due_date,
            "priority": priority,
            "notified": False,
            "completed": False
        }
        task_id_counter += 1
        return f"Task '{title}' added successfully!"
    except Exception as e:
        return f"Error: {str(e)}"

# Function to view tasks
def view_tasks(sort_by=None):
    if not tasks:
        return "No tasks found."
    sorted_tasks = list(tasks.items())
    if sort_by == "Priority":
        sorted_tasks = sorted(sorted_tasks, key=lambda x: x[1]["priority"])
    elif sort_by == "Deadline":
        sorted_tasks = sorted(sorted_tasks, key=lambda x: x[1]["due_date"])

    task_list = [
        f"ID: {task_id}, Title: {task['title']}, Due: {task['due_date']}, Priority: {task['priority']}, Completed: {'Yes' if task['completed'] else 'No'}"
        for task_id, task in sorted_tasks
    ]
    return "\n".join(task_list)

# Function to mark a task as completed
def mark_complete(task_id):
    try:
        task_id = int(task_id)
        if task_id in tasks:
            tasks[task_id]["completed"] = True
            return f"Task ID {task_id} marked as completed."
        else:
            return "Task not found."
    except ValueError:
        return "Invalid Task ID."

# Function to delete a task
def delete_task(task_id):
    try:
        task_id = int(task_id)
        if task_id in tasks:
            del tasks[task_id]
            return f"Task ID {task_id} deleted successfully."
        else:
            return "Task not found."
    except ValueError:
        return "Invalid Task ID."

# Function to send desktop notifications
def check_due_tasks():
    now = datetime.now()
    for task_id, task in tasks.items():
        if task["due_date"] <= now + timedelta(hours=1) and not task["notified"] and not task["completed"]:
            notification.notify(
                title="Task Reminder",
                message=f"Task '{task['title']}' is due in 1 hour!",
                timeout=10
            )
            task["notified"] = True

# Scheduler for notifications
scheduler = BackgroundScheduler()
scheduler.add_job(check_due_tasks, 'interval', minutes=1)
scheduler.start()

# Example usage (can be called from a different script, command line, etc.)
if __name__ == "__main__":
    # Add some tasks
    print(add_task("Complete PRP exam", "2025-01-26 14:00", "High"))
    print(add_task("Finish DCDS project", "2025-01-27 16:00", "Medium"))

    # View tasks
    print(view_tasks())

    # Mark a task as completed
    print(mark_complete(1))

    # Delete a task
    print(delete_task(2))

    # Run the scheduler (background job for notifications)
    while True:
        pass  # Keeps the script running


Task 'Complete PRP exam' added successfully!
Task 'Finish DCDS project' added successfully!
ID: 1, Title: Complete PRP exam, Due: 2025-01-26 14:00:00, Priority: High, Completed: No
ID: 2, Title: Finish DCDS project, Due: 2025-01-27 16:00:00, Priority: Medium, Completed: No
Task ID 1 marked as completed.
Task ID 2 deleted successfully.


KeyboardInterrupt: 

# AI writer

In [11]:
!pip install transformers

In [13]:
from transformers import pipeline
import pyotp
from cryptography.fernet import Fernet
from flask import Flask, render_template, request, redirect, url_for, session

# Initialize Flask app
app = Flask(__name__)
app.secret_key = 'your-secret-key'

# Encrypt and decrypt password using Fernet (symmetric encryption)
key = Fernet.generate_key()
cipher_suite = Fernet(key)

# Dummy user data (could be extended with database in a real-world scenario)
users = {'user1': 'encrypted_password'}

# Set up Hugging Face pipeline (using GPT-2 model for free)
generator = pipeline('text-generation', model='gpt2')

# Generate AI content based on user input
def generate_ai_content(topic):
    prompt = f"Generate a speech or lecture content on the topic: {topic}"
    result = generator(prompt, max_length=500, num_return_sequences=1)
    return result[0]['generated_text'].strip()

# Template selection for public/private speeches
def get_template(type_of_speech):
    templates = {
        'public': 'This is a public speech on {{ topic }}. The goal is to engage and motivate the audience...',
        'private': 'This is a private lecture for a select audience on {{ topic }}. The content should be detailed and factual...'
    }
    return templates.get(type_of_speech, 'Public speech template not found.')

# Route for home page
@app.route('/')
def home():
    return render_template('index.html')

# Route to generate speech content
@app.route('/generate', methods=['POST'])
def generate():
    topic = request.form['topic']
    type_of_speech = request.form['type_of_speech']

    # Generate AI content
    ai_content = generate_ai_content(topic)
    template = get_template(type_of_speech)

    # Combine AI content with the selected template
    speech = template.replace('{{ topic }}', topic) + '\n' + ai_content
    return render_template('speech_result.html', speech=speech)

# Route for login page (with password protection)
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Decrypt password and check if valid
        stored_password = users.get(username)
        if stored_password and cipher_suite.decrypt(stored_password.encode()).decode() == password:
            # 2FA step
            session['username'] = username
            return redirect(url_for('two_factor'))
        return "Invalid login"
    return render_template('login.html')

# Route for two-factor authentication
@app.route('/two_factor', methods=['GET', 'POST'])
def two_factor():
    if request.method == 'POST':
        otp = request.form['otp']
        totp = pyotp.TOTP('base32secret3232')  # Example secret
        if totp.verify(otp):
            return redirect(url_for('home'))
        return "Invalid OTP"

    # Generate a 2FA code
    totp = pyotp.TOTP('base32secret3232')
    otp = totp.now()
    return render_template('two_factor.html', otp=otp)

if __name__ == '__main__':
    app.run(debug=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
